In [5]:
import os
from pathlib import Path

path = Path('data/bodyfat_dataset.csv')


In [6]:
import torch, numpy as np, pandas as pd

df = pd.read_csv(path)

df

,title,url,meanPrediction,medianPrediction,bfPredictions,image_1,image_2,image_3,image_4,image_5
0,Leanest ive ever been. Never seen veins like t...,https://www.reddit.com/gallery/1b6k5jh,8.00,8.0,[8],https://preview.redd.it/leanest-ive-ever-been-...,https://preview.redd.it/leanest-ive-ever-been-...,https://preview.redd.it/leanest-ive-ever-been-...,https://preview.redd.it/leanest-ive-ever-been-...,NaN
1,Let me know. 78kg :),https://i.redd.it/4occeq9wdd2c1.jpg,9.80,10.0,"[9, 10, 10, 9, 11]",https://i.redd.it/4occeq9wdd2c1.jpg,NaN,NaN,NaN,NaN
2,What is my bf% I believe it’s around 13-14%,https://i.redd.it/lvfmowq0zhh91.jpg,14.33,13.5,"[13, 19, 14, 14, 13, 13]",https://i.redd.it/lvfmowq0zhh91.jpg,NaN,NaN,NaN,NaN
3,"25F | 4'11"" | 107 lbs",https://www.reddit.com/gallery/1e30z2f,23.33,24.0,"[20, 26, 24]",https://preview.redd.it/25f-411-107-lbs-v0-4lm...,https://preview.redd.it/25f-411-107-lbs-v0-huc...,NaN,NaN,NaN
4,Bodyfat?,https://www.reddit.com/gallery/1ktwao9,17.80,18.0,"[18, 16, 18, 15, 22]",https://preview.redd.it/8ekhv0d4xl2f1.jpg?widt...,https://preview.redd.it/rwkt50d4xl2f1.jpg?widt...,https://preview.redd.it/vau412d4xl2f1.jpg?widt...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
791,[GMBF] 6'1 220LBS Any estimates appreciated!,https://i.redd.it/z543jfquva731.jpg,12.00,12.0,[12],https://i.redd.it/z543jfquva731.jpg,NaN,NaN,NaN,NaN
792,"[GMBF] (M/22/6'2""/195lbs)",https://i.imgur.com/PY44dK5.jpg,10.50,10.5,"[11, 9, 10, 12]",https://i.imgur.com/PY44dK5.jpg,NaN,NaN,NaN,NaN
793,[GMBF] M/26/6'1/191lbs - down from 245lbs,https://i.imgur.com/pWGT7nV.jpg,15.00,15.0,[15],https://i.imgur.com/pWGT7nV.jpg,NaN,NaN,NaN,NaN
794,[GMBF] (M/26/5’10”/153lbs to 168lbs) What woul...,https://i.redd.it/lc2q2svumrt11.jpg,12.00,12.0,"[10, 14]",https://i.redd.it/lc2q2svumrt11.jpg,NaN,NaN,NaN,NaN


In [9]:
import requests
from tqdm import tqdm

import requests
import cv2
from PIL import Image
from tqdm import tqdm
import mediapipe as mp

def download_and_crop_images(df, output_dir="images"):
    os.makedirs(output_dir, exist_ok=True)
    image_cols = [f"image_{i}" for i in range(1, 6)]

    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

    def crop_upper_body_cv2(img):
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if not results.pose_landmarks:
            return None

        try:
            landmarks = results.pose_landmarks.landmark
            h, w, _ = img.shape

            xs = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w,
                  landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w,
                  landmarks[mp_pose.PoseLandmark.LEFT_HIP].x * w,
                  landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x * w]

            ys = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h,
                  landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h,
                  landmarks[mp_pose.PoseLandmark.LEFT_HIP].y * h,
                  landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y * h]

            xmin, xmax = int(min(xs)), int(max(xs))
            ymin, ymax = int(min(ys)), int(max(ys))

            # Add padding
            pad_x = 20
            pad_y = 20
            xmin = max(xmin - pad_x, 0)
            xmax = min(xmax + pad_x, w)
            ymin = max(ymin - pad_y, 0)
            ymax = min(ymax + pad_y, h)

            # Expand box
            expand = int((xmax - xmin) * 0.5)
            xmin = max(xmin - expand, 0)
            xmax = min(xmax + expand, w)

            cropped = img[ymin:ymax, xmin:xmax]
            return cropped
        except:
            return None

    seen_urls = set()
    image_count = 0
    failed = 0

    x = 0

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        for col in image_cols:
            x+=1
            url = row.get(col)
            if isinstance(url, str) and url.startswith("http") and url not in seen_urls:
                try:
                    response = requests.get(url, timeout=10)
                    if response.status_code == 200:
                        img_arr = np.asarray(bytearray(response.content), dtype=np.uint8)
                        img = cv2.imdecode(img_arr, cv2.IMREAD_COLOR)

                        if img is None:
                            raise ValueError("Downloaded image could not be decoded")

                        cropped = crop_upper_body_cv2(img)
                        final_img = cropped if cropped is not None else img
                        final_img = cv2.cvtColor(final_img, cv2.COLOR_BGR2RGB)
                        pil_img = Image.fromarray(final_img)

                        file_ext = url.split('.')[-1].split('?')[0]
                        file_name = f"{idx}_{col}.{file_ext}"
                        file_path = os.path.join(output_dir, file_name)
                        pil_img.save(file_path)

                        seen_urls.add(url)
                        image_count += 1
                    else:
                        failed += 1
                except Exception as e:
                    print(f"Error downloading or processing {url}: {e}")
                    failed += 1

    print(f"\n✔️ Downloaded and cropped {image_count} unique images to '{output_dir}/'")
    print(f"{x} images")
    if failed:
        print(f"⚠️ {failed} images failed to download or process.")


In [2]:
def create_regression_csv(df, output_csv="image_labels.csv", label_col="meanPrediction", image_prefix="image_"):
    df.columns = df.columns.str.strip()
    image_cols = [col for col in df.columns if col.startswith(image_prefix)]
    records = []

    for idx, row in df.iterrows():
        label = row[label_col]

        for col in image_cols:
            url = row.get(col)

            # Debug url at idx 176 for each image col
            if idx == 175:
                print(f"DEBUG idx={idx} col={col} url={url}")

            if isinstance(url, str) and url.startswith("http"):
                ext = url.split('.')[-1].split('?')[0].lower()
                ext = ext if ext in ['jpg', 'jpeg', 'png', 'webp'] else 'jpg'
                filename = f"{idx}_{col}.{ext}"

                # Debug filename and label for idx 176
                if idx == 176:
                    print(f"DEBUG idx={idx} filename={filename} label={label}")

                records.append({"filename": filename, "target": label})

    df_out = pd.DataFrame(records)
    df_out.to_csv(output_csv, index=False)
    print(f"Created {output_csv} with {len(df_out)} labeled images")
    return df_out


In [3]:
download_and_crop_images(df)  

NameError: name 'download_and_crop_images' is not defined

In [7]:
df_labels = create_regression_csv(df, output_csv="data/image_labels.csv") 
df_labels


DEBUG idx=175 col=image_1 url=https://preview.redd.it/6-172-lbs-curious-about-bf-after-losing-280-lbs-v0-jz3w2jzvq96e1.jpg?width=640&crop=smart&auto=webp&s=27969c92649102cfd44ee367f269f92ee7b69131
DEBUG idx=175 col=image_2 url=https://preview.redd.it/6-172-lbs-curious-about-bf-after-losing-280-lbs-v0-cib4qizvq96e1.jpg?width=640&crop=smart&auto=webp&s=059bc43b751154ec72149fce338e4169173dfbec
DEBUG idx=175 col=image_3 url=https://preview.redd.it/6-172-lbs-curious-about-bf-after-losing-280-lbs-v0-3b9oyizvq96e1.jpg?width=640&crop=smart&auto=webp&s=1d52db25d8c6961213364ced376cc4afc75bedce
DEBUG idx=175 col=image_4 url=https://preview.redd.it/6-172-lbs-curious-about-bf-after-losing-280-lbs-v0-6b31hjzvq96e1.jpg?width=640&crop=smart&auto=webp&s=4c9720f857a9a71edc268d1aab4d32554f1c646a
DEBUG idx=175 col=image_5 url=nan
DEBUG idx=176 filename=176_image_1.jpg label=15.67
Created data/image_labels.csv with 1580 labeled images


,filename,target
0,0_image_1.jpg,8.0
1,0_image_2.jpg,8.0
2,0_image_3.jpg,8.0
3,0_image_4.jpg,8.0
4,1_image_1.jpg,9.8
...,...,...
1575,791_image_1.jpg,12.0
1576,792_image_1.jpg,10.5
1577,793_image_1.jpg,15.0
1578,794_image_1.jpg,12.0


In [8]:
from fastai.vision.all import *

failed = verify_images(get_image_files(Path('images')))
failed.map(Path.unlink)
len(failed)

0

In [9]:
from sklearn.model_selection import train_test_split

initial_nan_labels_count = df_labels['target'].isna().sum()
if initial_nan_labels_count > 0:
    print(f"DEBUG: Found {initial_nan_labels_count} NaN values in 'target' column. Dropping rows with NaN targets.")
    df_labels.dropna(subset=['target'], inplace=True)
else:
    print("DEBUG: No NaN values found in 'target' column.")

train_df, valid_df = train_test_split(df_labels, test_size=0.2, random_state=42)

train_df.to_csv("data/train_set.csv")
valid_df.to_csv("data/valid_set.csv")

train_label_dict = dict(zip(train_df['filename'], train_df['target']))
valid_label_dict = dict(zip(valid_df['filename'], valid_df['target']))
all_label_dict = {**train_label_dict, **valid_label_dict}

DEBUG: No NaN values found in 'target' column.
